In [1]:
import pandas as pd # Обновить pandas до последней версии
import numpy as np
import math
import os
import glob
import mysql.connector
import datetime
from mysql.connector import errorcode
from sklearn import preprocessing
from sklearn import metrics

import matplotlib.pyplot as plt
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
import string


import warnings
warnings.filterwarnings("ignore")

In [2]:
user = 'root'
password = '12345'

## Пункт 1. 

In [3]:
cnx = mysql.connector.connect(host = 'localhost', database = 'hw_db', user = user, password = password)
cursor = cnx.cursor()

payments_table = pd.read_sql_query("""SELECT contracts.contract_number, term_month, date, amount_due,amount_paid
FROM contracts 
INNER JOIN payments on contracts.contract_number = payments.contract_number""", cnx)
payments_table.keys()  #выгружаем из БД часть данных из таблиц с платежами и контрактами

Index(['contract_number', 'term_month', 'date', 'amount_due', 'amount_paid'], dtype='object')

In [4]:
contract_number = list(payments_table['contract_number'])
contract_number_unique = list(set(contract_number)) #получаем список номеров контрактов
print (contract_number_unique)

[100737, 100610, 100869, 100614, 100997, 100743, 100872, 100620, 100493, 100368, 100113, 100375, 100122, 100506, 100380, 100765, 100127, 100516, 100901, 100518, 101028, 100520, 100648, 100856, 100524, 100022, 100668, 100797, 100414, 100540, 100929, 100932, 100933, 100552, 100297, 100684, 100431, 100176, 100303, 100689, 100052, 100441, 100190, 100703, 100320, 100321, 100958, 100323, 100068, 100582, 100455, 100585, 100206, 100847, 100851, 100211, 100468, 100600, 100476, 100990]


In [5]:
risk_horizon = 12

In [6]:
default_info_with_risk_horizon_12 = [] #лист дефолтов на горизонте год

In [7]:
for i in contract_number_unique:
    table = payments_table[payments_table['contract_number'] == i]
    if table.iloc[0][1] >= risk_horizon:
        table = table[:risk_horizon]
        table = table.reset_index()
        for j in range(0, table.shape[0]-3):
            if (table['amount_paid'][j] < table['amount_due'][j] and table['amount_paid'][j+1] < table['amount_due'][j+1] and
                table['amount_paid'][j+2] < table['amount_due'][j+2] and table['amount_paid'][j+3] < table['amount_due'][j+3]):
                #проверяем, что заемщик не платит 4 даты подряд
                default_date = table['date'][j + 3] #фиксируем дату просрочки
                default_date = (str(pd.to_datetime(default_date).year) + '-' + str(pd.to_datetime(default_date).month) + 
                                '-' + str(pd.to_datetime(default_date).day)) #корректируем формат даты просрочки
                while (i, '0000-00-00', 0) in default_info_with_risk_horizon_12:
                                default_info_with_risk_horizon_12.remove ((i, '0000-00-00', 0)) 
                        #исключаем из нашего листа дефолтов нулевые даты для дефолтеров
                default_info_with_risk_horizon_12.append((i, default_date, 1)) 
                break 
            else: default_info_with_risk_horizon_12.append((i, '0000-00-00', 0))
    
default_info_with_risk_horizon_12[:5]

[(100737, '0000-00-00', 0),
 (100737, '0000-00-00', 0),
 (100737, '0000-00-00', 0),
 (100737, '0000-00-00', 0),
 (100737, '0000-00-00', 0)]

In [8]:
default_info_with_risk_horizon_12 = list(set(default_info_with_risk_horizon_12)) #отбросим повторяющиеся значения
print (len(default_info_with_risk_horizon_12))
default_info_with_risk_horizon_12 

60


[(100620, '0000-00-00', 0),
 (100933, '0000-00-00', 0),
 (100610, '0000-00-00', 0),
 (100737, '0000-00-00', 0),
 (100206, '0000-00-00', 0),
 (100516, '0000-00-00', 0),
 (101028, '0000-00-00', 0),
 (100122, '0000-00-00', 0),
 (100506, '0000-00-00', 0),
 (100797, '0000-00-00', 0),
 (100320, '0000-00-00', 0),
 (100518, '0000-00-00', 0),
 (100703, '0000-00-00', 0),
 (100380, '0000-00-00', 0),
 (100856, '0000-00-00', 0),
 (100689, '0000-00-00', 0),
 (100414, '0000-00-00', 0),
 (100600, '0000-00-00', 0),
 (100493, '0000-00-00', 0),
 (100958, '0000-00-00', 0),
 (100585, '0000-00-00', 0),
 (100022, '0000-00-00', 0),
 (100847, '0000-00-00', 0),
 (100552, '0000-00-00', 0),
 (100321, '0000-00-00', 0),
 (100765, '0000-00-00', 0),
 (100375, '2017-1-2', 1),
 (100614, '0000-00-00', 0),
 (100476, '0000-00-00', 0),
 (100368, '0000-00-00', 0),
 (100297, '0000-00-00', 0),
 (100431, '0000-00-00', 0),
 (100684, '0000-00-00', 0),
 (100869, '0000-00-00', 0),
 (100997, '0000-00-00', 0),
 (100303, '0000-00-00'

In [9]:
cnx = mysql.connector.connect(host = 'localhost', database = 'hw_db', user = user, password = password)
cursor = cnx.cursor()

cursor.execute("""CREATE TABLE IF NOT EXISTS contracts_risk_horizon_{0}_months 
(contract_number INT, 
bankruptcy BOOLEAN, 
date_of_default DATE)""".format(risk_horizon))

cnx.commit()
cnx.close()

In [10]:
cnx = mysql.connector.connect(host = 'localhost', database = 'hw_db', user = user, password = password)
cursor = cnx.cursor()

for i in default_info_with_risk_horizon_12:
    cursor.execute("""INSERT INTO contracts_risk_horizon_{0}_months 
    VALUES
    (%d, %d, '%s')""".format(risk_horizon) % (
    i[0],
    i[2],
    i[1]))
    
cnx.commit()
cnx.close()

In [11]:
cnx = mysql.connector.connect(host = 'localhost', database = 'hw_db', user = user, password = password)
cursor = cnx.cursor()

applications_with_risk_horizon_12 = pd.read_sql_query("""SELECT bankruptcy, age, age_of_car, income, children, family, 
gender, marital_status_enc, employed_by_enc, education_enc, position_enc, income_type_enc, housing_enc, 
house_ownership_enc, amount, term_month FROM
(SELECT bankruptcy, id, amount, term_month
FROM contracts
INNER JOIN contracts_risk_horizon_{0}_months ON
contracts.contract_number = contracts_risk_horizon_{1}_months.contract_number) AS A
INNER JOIN applications ON
applications.id = A.id""".format(risk_horizon, risk_horizon), cnx)

In [12]:
applications_with_risk_horizon_12

,bankruptcy,age,age_of_car,income,children,family,gender,marital_status_enc,employed_by_enc,education_enc,position_enc,income_type_enc,housing_enc,house_ownership_enc,amount,term_month
0,0,48,NaN,112500.0,1,3,1,1,8,3,6,4,0,2,327024,17
1,0,35,13.0,76500.0,0,2,1,1,4,3,0,4,0,0,454500,34
2,0,71,NaN,83250.0,0,2,0,1,25,3,0,2,0,2,239850,13
3,0,32,NaN,225000.0,2,4,0,1,17,0,4,0,0,2,314055,22
4,0,34,NaN,112500.0,0,2,0,0,18,3,9,4,0,2,512064,23
5,0,57,NaN,99000.0,0,2,0,1,1,3,8,4,0,2,260640,13
6,0,54,NaN,81000.0,0,2,1,1,15,3,11,4,0,2,247500,23
7,0,33,NaN,90000.0,1,2,0,4,9,0,4,3,0,2,544491,34
8,0,30,1.0,270000.0,0,1,1,4,11,0,0,3,0,2,790830,17
9,0,55,NaN,121500.0,0,1,1,4,13,3,4,4,0,2,513000,26


## Пункт 2.

In [13]:
defaults = applications_with_risk_horizon_12.bankruptcy #выведем флаги на горизонте год
defaults

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    1
27    0
28    0
29    0
30    0
31    0
32    0
33    0
34    0
35    0
36    0
37    0
38    0
39    0
40    0
41    1
42    0
43    0
44    0
45    0
46    0
47    0
48    0
49    0
50    0
51    0
52    0
53    0
54    0
55    0
56    0
57    0
58    0
59    0
Name: bankruptcy, dtype: int64

In [14]:
applications_with_risk_horizon_12 = applications_with_risk_horizon_12.drop(['bankruptcy'], axis = 1)

##### Построим WOE и IV

In [15]:
max_bin = 20
force_bin = 4

# define a binning function
def char_bin(Y, X):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]    
    df2 = notmiss.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.sum().Y.index
    d3["MAX_VALUE"] = d3["MIN_VALUE"]
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return(d3)

def woe_iv_categorical(df1, target):
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        if i.upper() not in (final.upper()):
            conv = char_bin(target, df1[i])
            conv["VAR_NAME"] = i
            count = count + 1
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return(iv_df,iv)

In [16]:
applications_with_risk_horizon_12_1  = applications_with_risk_horizon_12.copy()

##### Группируем переменные на этой основе

In [17]:
applications_with_risk_horizon_12_1['age'].describe()

count    60.000000
mean     45.316667
std      12.155060
min      21.000000
25%      34.750000
50%      46.500000
75%      54.250000
max      71.000000
Name: age, dtype: float64

In [18]:
applications_with_risk_horizon_12_1['age_of_car'].describe()

count    16.000000
mean     10.312500
std       7.551766
min       1.000000
25%       3.000000
50%       9.000000
75%      14.750000
max      26.000000
Name: age_of_car, dtype: float64

In [19]:
applications_with_risk_horizon_12_1['income'].describe()

count        60.000000
mean     171677.820000
std      113656.310989
min       38419.200000
25%       96750.000000
50%      126000.000000
75%      202500.000000
max      540000.000000
Name: income, dtype: float64

##### Переделаем в категориальные переменные

In [20]:
applications_with_risk_horizon_12_1['age'] = np.where( (applications_with_risk_horizon_12_1['age'] <= 34) & (applications_with_risk_horizon_12_1['age'] >= 21), 0, 
                               np.where ((applications_with_risk_horizon_12_1['age'] <= 46) & (applications_with_risk_horizon_12_1['age'] > 34), 1,
                                        np.where ((applications_with_risk_horizon_12_1['age'] <= 54) & (applications_with_risk_horizon_12_1['age'] > 46), 2, 3)))

In [21]:
applications_with_risk_horizon_12_1['age_of_car'] = np.where( (applications_with_risk_horizon_12_1['age_of_car'] <= 3) & (applications_with_risk_horizon_12_1['age_of_car'] >= 1), 0, 
                               np.where ((applications_with_risk_horizon_12_1['age_of_car'] <= 9) & (applications_with_risk_horizon_12_1['age_of_car'] > 3), 1,
                                        np.where ((applications_with_risk_horizon_12_1['age_of_car'] <= 15) & (applications_with_risk_horizon_12_1['age_of_car'] > 9), 2, 3)))

In [22]:
applications_with_risk_horizon_12_1['income'] = np.where( (applications_with_risk_horizon_12_1['income'] <= 96750) & (applications_with_risk_horizon_12_1['income'] >= 38419), 0, 
                               np.where ((applications_with_risk_horizon_12_1['income'] <= 126000) & (applications_with_risk_horizon_12_1['income'] > 96750), 1,
                                        np.where ((applications_with_risk_horizon_12_1['income'] <= 202500) & (applications_with_risk_horizon_12_1['income'] > 126000), 2, 3)))

In [23]:
applications_with_risk_horizon_12_1 = applications_with_risk_horizon_12_1.drop(['amount'], axis = 1)

In [24]:
applications_with_risk_horizon_12_1 = applications_with_risk_horizon_12_1.drop(['term_month'], axis = 1)

In [25]:
applications_with_risk_horizon_12_1 = applications_with_risk_horizon_12_1.drop(['age'], axis = 1)

In [26]:
applications_with_risk_horizon_12_1 = applications_with_risk_horizon_12_1.drop(['age_of_car'], axis = 1)

In [27]:
applications_with_risk_horizon_12_1 = applications_with_risk_horizon_12_1.drop(['income'], axis = 1)

In [28]:
applications_with_risk_horizon_12_1 = applications_with_risk_horizon_12_1.drop(['family'], axis = 1)

In [29]:
applications_with_risk_horizon_12_1 = applications_with_risk_horizon_12_1.drop(['employed_by_enc'], axis = 1)

In [30]:
applications_with_risk_horizon_12_1 = applications_with_risk_horizon_12_1.drop(['position_enc'], axis = 1)

In [31]:
applications_with_risk_horizon_12_1.head(5) #таблица переменных в категориях

,children,gender,marital_status_enc,education_enc,income_type_enc,housing_enc,house_ownership_enc
0,1,1,1,3,4,0,2
1,0,1,1,3,4,0,0
2,0,0,1,3,2,0,2
3,2,0,1,0,0,0,2
4,0,0,0,3,4,0,2


In [32]:
woe_iv_cat_df = woe_iv_categorical(applications_with_risk_horizon_12_1, defaults)[0]
woe_iv_cat_df

,VAR_NAME,MIN_VALUE,MAX_VALUE,COUNT,EVENT,EVENT_RATE,NONEVENT,NON_EVENT_RATE,DIST_EVENT,DIST_NON_EVENT,WOE,IV
0,children,0,0,47,2,0.042553,45,0.957447,1.0,0.775862,0.253781,0.056882
1,children,1,1,12,0,0.000000,12,1.000000,0.0,0.206897,0.000000,0.056882
2,children,2,2,1,0,0.000000,1,1.000000,0.0,0.017241,0.000000,0.056882
3,gender,0,0,35,2,0.057143,33,0.942857,1.0,0.568966,0.563935,0.243076
4,gender,1,1,25,0,0.000000,25,1.000000,0.0,0.431034,0.000000,0.243076
5,marital_status_enc,0,0,8,1,0.125000,7,0.875000,0.5,0.120690,1.421386,0.839995
6,marital_status_enc,1,1,36,0,0.000000,36,1.000000,0.0,0.620690,0.000000,0.839995
7,marital_status_enc,3,3,1,0,0.000000,1,1.000000,0.0,0.017241,0.000000,0.839995
8,marital_status_enc,4,4,12,1,0.083333,11,0.916667,0.5,0.189655,0.969401,0.839995
9,marital_status_enc,5,5,3,0,0.000000,3,1.000000,0.0,0.051724,0.000000,0.839995


In [41]:
applications_with_risk_horizon_12_1['gender_score'] = [10 if x  == 1 else 2 for x in applications_with_risk_horizon_12_1['gender']]

In [34]:
applications_with_risk_horizon_12_1['marital_status_score'] = np.where(applications_with_risk_horizon_12_1['marital_status_enc'] == 5, 10, 
                                          np.where(applications_with_risk_horizon_12_1['marital_status_enc'] == 3, 6, 2))

In [35]:
applications_with_risk_horizon_12_1['education_score'] = np.where(applications_with_risk_horizon_12_1['education_enc'] == 0, 2, 
                                          np.where(applications_with_risk_horizon_12_1['education_enc'] == 1, 6, 10))

In [36]:
applications_with_risk_horizon_12_1['income_type_score'] = np.where(applications_with_risk_horizon_12_1['income_type_enc'] == 2, 6, 
                                          np.where(applications_with_risk_horizon_12_1['income_type_enc'] == 4, 8, 4))

In [37]:
applications_with_risk_horizon_12_1['house_ownership_score'] = np.where(applications_with_risk_horizon_12_1['house_ownership_enc'] == 2, 8, 
                                          np.where(applications_with_risk_horizon_12_1['house_ownership_enc'] == 0, 4, 10))

In [38]:
applications_with_risk_horizon_12_1['children_score'] = np.where(applications_with_risk_horizon_12_1['children'] == 2, 6, 
                                          np.where(applications_with_risk_horizon_12_1['children'] == 0, 8, 4))

In [39]:
applications_with_risk_horizon_12_1

,children,gender,marital_status_enc,education_enc,income_type_enc,housing_enc,house_ownership_enc,marital_status_score,education_score,income_type_score,house_ownership_score,children_score
0,1,1,1,3,4,0,2,2,10,8,8,4
1,0,1,1,3,4,0,0,2,10,8,4,8
2,0,0,1,3,2,0,2,2,10,6,8,8
3,2,0,1,0,0,0,2,2,2,4,8,6
4,0,0,0,3,4,0,2,2,10,8,8,8
5,0,0,1,3,4,0,2,2,10,8,8,8
6,0,1,1,3,4,0,2,2,10,8,8,8
7,1,0,4,0,3,0,2,2,2,4,8,4
8,0,1,4,0,3,0,2,2,2,4,8,8
9,0,1,4,3,4,0,2,2,10,8,8,8


In [40]:
cnx.close()